Name: Saral Gautam
Student ID: 30618428
email: sgau0007@student.monash.edu

# FIT3182 Assignment 1: Part A

Task 1:
The data model will follow the document data model. This is because the data contains multiple different types for example datetime (in hotspot_historic), date, integers and strings (used in preciptation measurements), which the document allows for the easy storage of. Additionally the data is relatively unstructured- in climate historic each set of measurements is taken daily without any kind of grouping between the measurements and in the hotspot data there is no sorting with any datetimes. Furthermore since the dates are common between both the hotspot_historic and climate_historic we have at least one attribute that we can use to group the attributes from both files together into a document and have overall denormalised data. This is optimal since we only plan to run search queries through the data which will be much faster when the documents are grouped together in this way. The document data model also allows for some attributes to be empty and this may be the case since one climate data may have zero or many fire data, hence leading to some possibly empty attributes.
Example of data with the types for each attribute:
If there is a fire: {
             "latitude" : float,
             "longitude": float,
             "datetime" : string,
             "confidence" : float,
             "surface_temperature_celsius" : float,
            "station": float,
            "date": string,
            "air_temperature_celsius" : float,
            "relative_humidity": float,
            "windspeed_knots" : float,
            "max_wind_speed": float,
            "precipitation_level": float,
            "precipitation_type": string,
            "GHI_w/m2": float}
            
If there is no fire:  {
            "latitude" : None,
            "longitude": None,
            "datetime" : None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "station": int,
            "date": string,
            "air_temperature_celsius" : float
            "relative_humidity": float,
            "windspeed_knots" : float,
            "max_wind_speed":float,
            "precipitation_level": float,
            "precipitation_type": string,
            "GHI_w/m2": float
        }


In [3]:
from pprint import pprint
import pymongo
from pymongo import MongoClient
import csv

In [4]:
def samedate(hotspot_date, climate_date):
    # this function checks if two dates are the same
    # This function was made because date is forrmatted differently in hotspot_historic and climate_historic
    # eg. hotspot historic '27/12/2018' and climate historic '27/12/18'
    actual_date = []
    final_date = []
    join_string = ''
    date_length = len(hotspot_date)
    for k in range(date_length - 1, -1, -1):
        if k == 7 or k == 6:
            continue
        else:
            actual_date.append(hotspot_date[k])
    for j in range(date_length - 3, -1, -1):
        final_date.append(actual_date[j])

    return (join_string.join(final_date) == climate_date)

In [5]:
def get_prec_number(a_string):
    # this function gives the float part of a precipitation measurement
    final = a_string.strip()
    return float(final[0:4])
    

def get_prec_letter(a_string):
    # this function gives the letter representing the tyoe of precipitation measurement
    final = a_string.strip()
    return final[-1]

In [6]:
# Task 2
# Question 1
try:  # this try and except clause sees if the_collection has already been made. If so we will get rid of it.
    the_collection.drop() 
except NameError:
    pass

# Now create the data base
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
the_collection = db.climate  # create a new collection called 'the_collection'
with open('climate_historic.csv', 'r') as file_one:  # open climate historic file
    # now create a csv object for each of the files
    climate_data = csv.reader(file_one)
    climate_lines = []
    count = 0
    for c_line in climate_data:
        if count == 0:
            count += 1
        else:
            climate_lines.append(c_line)   # put each line of the data in a list 
            

            
with open('hotspot_historic.csv', 'r') as file_two: # open hotspot historic file
    hotspot_data = csv.reader(file_two)
    hotspot_lines = []
    count = 0
    for h_line in hotspot_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            hotspot_lines.append(h_line)  # put each line of the data in a list

documents = []
for climate_line in climate_lines:  # iterate through all the lines in the climate_historic.csv
    matched = False
    for hotspot_line in hotspot_lines:  # iterate through all the lines in hotspot_historic.csv
        if samedate(hotspot_line[4], climate_line[1]):  # if a climate data has a fire
            # then make this data entry
            new_entry = {
             "latitude" : float(hotspot_line[0]),
             "longitude": float(hotspot_line[1]),
             "datetime" : hotspot_line[2],
             "confidence" : float(hotspot_line[3]),
             "surface_temperature_celsius" : float(hotspot_line[5]),
            "station": float(climate_line[0]),
            "date": climate_line[1],
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])}
            documents.append(new_entry)  # put this entry into a documents list
            matched = True
    if matched == False: # there was not a fire on this date
        # Set None's for all the unmatchable values cause there was no fire on this date
        no_fire = {
            "latitude" : None,
            "longitude": None,
            "datetime" : None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "station": float(climate_line[0]),
            "date": climate_line[1],
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])
        }
        documents.append(no_fire)  # put the entry into the documents list


result = the_collection.insert_many(documents)   # insert all the documents into the collection



Below are all the queries for Task 2 Question 2

In [7]:
# a) Here we run a simple query with the date set the December 12th 2018. Since we are told the date directly we
# can directly search for it. Also we can format the data using "attribute": 1 to only get the climate data.
pprint(the_collection.find_one({"date": "12/12/18"}, 
                               {"date": 1, "air_temperature_celsius" : 1, "relative_humidity": 1, "windspeed_knots": 1,
                                "max_wind_speed": 1, "precipitation": 1, "GHI_w/m2": 1
                               }))

{'GHI_w/m2': 156.0,
 '_id': ObjectId('60aa1920c946a2389d1839c4'),
 'air_temperature_celsius': 19.0,
 'date': '12/12/18',
 'max_wind_speed': 12.0,
 'relative_humidity': 55.3,
 'windspeed_knots': 6.2}


In [8]:
# b)  Here we can use a range based query with the $and, then format the data with "attribute" : 1 
answer = the_collection.find({"$and": [
    {"surface_temperature_celsius": {"$gte": 65}},
    {"surface_temperature_celsius": {"$lte": 100}}   
]},  
{"latitude": 1, "longitude": 1, "surface_temperature_celsius": 1, "confidence": 1}
                             
                             
)

for item in answer:
    pprint(item)

{'_id': ObjectId('60aa1920c946a2389d183934'),
 'confidence': 96.0,
 'latitude': -38.127,
 'longitude': 143.82,
 'surface_temperature_celsius': 77.0}
{'_id': ObjectId('60aa1920c946a2389d18393a'),
 'confidence': 100.0,
 'latitude': -37.294,
 'longitude': 141.232,
 'surface_temperature_celsius': 87.0}
{'_id': ObjectId('60aa1920c946a2389d18393b'),
 'confidence': 99.0,
 'latitude': -37.293,
 'longitude': 141.245,
 'surface_temperature_celsius': 85.0}
{'_id': ObjectId('60aa1920c946a2389d183940'),
 'confidence': 91.0,
 'latitude': -37.247,
 'longitude': 141.278,
 'surface_temperature_celsius': 68.0}
{'_id': ObjectId('60aa1920c946a2389d183950'),
 'confidence': 92.0,
 'latitude': -36.098,
 'longitude': 143.74,
 'surface_temperature_celsius': 67.0}
{'_id': ObjectId('60aa1920c946a2389d183975'),
 'confidence': 89.0,
 'latitude': -36.94,
 'longitude': 143.281,
 'surface_temperature_celsius': 69.0}
{'_id': ObjectId('60aa1920c946a2389d18397b'),
 'confidence': 87.0,
 'latitude': -37.862,
 'longitude':

In [14]:
# c) Here we can run two queries for each date and then format the data so we have the required information.
# which in this question was the date, surface temperature, air temperature, relative humidity and max wind speed.
fifteenth = the_collection.find({"date": '15/12/18'}, {"date": 1, "surface_temperature_celsius": 1, "air_temperature_celsius": 1, 
                             "relative_humidity": 1, "max_wind_speed": 1
                             })
for item in fifteenth:
    pprint(item)

sixteenth = the_collection.find({"date": '16/12/18'}, {"date": 1, "surface_temperature_celsius": 1, "air_temperature_celsius": 1, 
                             "relative_humidity": 1, "max_wind_speed": 1
                             })
for item in sixteenth:
    pprint(item)


{'_id': ObjectId('60aa1920c946a2389d1839c7'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 42.0}
{'_id': ObjectId('60aa1920c946a2389d1839c8'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 36.0}
{'_id': ObjectId('60aa1920c946a2389d1839c9'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 38.0}
{'_id': ObjectId('60aa1920c946a2389d1839ca'),
 'air_temperature_celsius': 18.0,
 'date': '15/12/18',
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celsius': 40.0}
{'_id': ObjectId('60aa1920c946a2389d1839cb'),
 'air_temperature_celsius': 18.0,
 'date': '16/12/18',
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celsius': 43.0}
{'_id': ObjectId('60aa1920c946a2389d1839

In [15]:
# d) Here is another range based query like question b
d_answer = the_collection.find({"$and": [
    {"confidence": {"$gte": 80}},
    {"confidence": {"$lte": 100}}   
]},  
{"datetime": 1, "air_temperature_celsius": 1, "surface_temperature_celsius": 1, "confidence": 1}
                             
                             
)
for item in d_answer:
    pprint(item)

{'_id': ObjectId('60aa1920c946a2389d183934'),
 'air_temperature_celsius': 13.0,
 'confidence': 96.0,
 'datetime': '2018-10-10T04:04:19',
 'surface_temperature_celsius': 77.0}
{'_id': ObjectId('60aa1920c946a2389d18393a'),
 'air_temperature_celsius': 13.0,
 'confidence': 100.0,
 'datetime': '2018-10-15T04:23:01',
 'surface_temperature_celsius': 87.0}
{'_id': ObjectId('60aa1920c946a2389d18393b'),
 'air_temperature_celsius': 13.0,
 'confidence': 99.0,
 'datetime': '2018-10-15T04:23:01',
 'surface_temperature_celsius': 85.0}
{'_id': ObjectId('60aa1920c946a2389d18393e'),
 'air_temperature_celsius': 22.0,
 'confidence': 94.0,
 'datetime': '2018-10-17T13:36:12',
 'surface_temperature_celsius': 43.0}
{'_id': ObjectId('60aa1920c946a2389d183940'),
 'air_temperature_celsius': 22.0,
 'confidence': 91.0,
 'datetime': '2018-10-17T04:10:45',
 'surface_temperature_celsius': 68.0}
{'_id': ObjectId('60aa1920c946a2389d183942'),
 'air_temperature_celsius': 22.0,
 'confidence': 82.0,
 'datetime': '2018-10-1

In [16]:
# e) Here we can sort the surface temperature and then find the first ten entries using .limit()
e_answer = the_collection.find().sort("surface_temperature_celsius", pymongo.DESCENDING).limit(10)
for item in e_answer:
    pprint(item)

{'GHI_w/m2': 111.0,
 '_id': ObjectId('60aa1920c946a2389d18393a'),
 'air_temperature_celsius': 13.0,
 'confidence': 100.0,
 'date': '15/10/18',
 'datetime': '2018-10-15T04:23:01',
 'latitude': -37.294,
 'longitude': 141.232,
 'max_wind_speed': 13.0,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 50.7,
 'station': 948702.0,
 'surface_temperature_celsius': 87.0,
 'windspeed_knots': 6.1}
{'GHI_w/m2': 227.0,
 '_id': ObjectId('60aa1920c946a2389d183995'),
 'air_temperature_celsius': 28.0,
 'confidence': 100.0,
 'date': '29/11/18',
 'datetime': '2018-11-29T03:52:10',
 'latitude': -37.606,
 'longitude': 149.312,
 'max_wind_speed': 16.9,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 56.7,
 'station': 948702.0,
 'surface_temperature_celsius': 87.0,
 'windspeed_knots': 9.3}
{'GHI_w/m2': 202.0,
 '_id': ObjectId('60aa1920c946a2389d1839ad'),
 'air_temperature_celsius': 26.0,
 'confidence': 99.0,
 'date': '30/11/18',
 'datetime': '2018-1

In [17]:
# f) Here we can find the entries where there are entries for latitude. For my data model this means that there
# was a fire. Then we count the number of fires on that date.
# if there is no date then no fire occured that day i.e count = 0
f_answer = the_collection.aggregate([ 

    {"$match": {"latitude": {"$ne": None}}},
    {"$group": {"_id": "$date", "count": {"$sum": 1}}}

])
for item in f_answer:
    pprint(item)


{'_id': '27/12/18', 'count': 4}
{'_id': '25/12/18', 'count': 1}
{'_id': '24/12/18', 'count': 1}
{'_id': '16/12/18', 'count': 15}
{'_id': '14/12/18', 'count': 1}
{'_id': '13/12/18', 'count': 1}
{'_id': '12/12/18', 'count': 1}
{'_id': '21/10/18', 'count': 4}
{'_id': '27/10/18', 'count': 5}
{'_id': '15/10/18', 'count': 3}
{'_id': '21/12/18', 'count': 1}
{'_id': '23/10/18', 'count': 1}
{'_id': '20/10/18', 'count': 3}
{'_id': '12/11/18', 'count': 5}
{'_id': '13/11/18', 'count': 5}
{'_id': '16/10/18', 'count': 1}
{'_id': '23/11/18', 'count': 5}
{'_id': '30/11/18', 'count': 31}
{'_id': '10/10/18', 'count': 3}
{'_id': '28/10/18', 'count': 1}
{'_id': '22/11/18', 'count': 2}
{'_id': '17/10/18', 'count': 5}
{'_id': '21/11/18', 'count': 1}
{'_id': '15/12/18', 'count': 4}
{'_id': '18/10/18', 'count': 6}
{'_id': '11/11/18', 'count': 4}
{'_id': '28/11/18', 'count': 1}
{'_id': '26/10/18', 'count': 5}
{'_id': '14/11/18', 'count': 3}
{'_id': '29/11/18', 'count': 8}
{'_id': '10/12/18', 'count': 2}


In [18]:
# g) Here we filter for the entries that had fires and then find the ones where the confidence is below 70.
g_answer = the_collection.find({"$and": [{"latitude": {"$ne": None}},     {"confidence": {"$lt": 70}} ]})

for item in g_answer:
    pprint(item)

{'GHI_w/m2': 157.0,
 '_id': ObjectId('60aa1920c946a2389d18393d'),
 'air_temperature_celsius': 19.0,
 'confidence': 62.0,
 'date': '16/10/18',
 'datetime': '2018-10-16T23:56:43',
 'latitude': -37.288,
 'longitude': 144.39,
 'max_wind_speed': 12.0,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 54.3,
 'station': 948702.0,
 'surface_temperature_celsius': 36.0,
 'windspeed_knots': 5.9}
{'GHI_w/m2': 194.0,
 '_id': ObjectId('60aa1920c946a2389d183941'),
 'air_temperature_celsius': 22.0,
 'confidence': 58.0,
 'date': '17/10/18',
 'datetime': '2018-10-17T04:10:38',
 'latitude': -37.587,
 'longitude': 142.479,
 'max_wind_speed': 16.9,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 46.8,
 'station': 948702.0,
 'surface_temperature_celsius': 41.0,
 'windspeed_knots': 8.0}
{'GHI_w/m2': 119.0,
 '_id': ObjectId('60aa1920c946a2389d18394c'),
 'air_temperature_celsius': 13.0,
 'confidence': 65.0,
 'date': '20/10/18',
 'datetime': '2018-10-2

 '_id': ObjectId('60aa1920c946a2389d1839c9'),
 'air_temperature_celsius': 18.0,
 'confidence': 51.0,
 'date': '15/12/18',
 'datetime': '2018-12-15T13:17:17',
 'latitude': -37.623,
 'longitude': 149.323,
 'max_wind_speed': 14.0,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 52.0,
 'station': 948702.0,
 'surface_temperature_celsius': 38.0,
 'windspeed_knots': 7.1}
{'GHI_w/m2': 152.0,
 '_id': ObjectId('60aa1920c946a2389d1839ca'),
 'air_temperature_celsius': 18.0,
 'confidence': 65.0,
 'date': '15/12/18',
 'datetime': '2018-12-15T01:16:23',
 'latitude': -38.038,
 'longitude': 142.986,
 'max_wind_speed': 14.0,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 52.0,
 'station': 948702.0,
 'surface_temperature_celsius': 40.0,
 'windspeed_knots': 7.1}
{'GHI_w/m2': 150.0,
 '_id': ObjectId('60aa1920c946a2389d1839cc'),
 'air_temperature_celsius': 18.0,
 'confidence': 69.0,
 'date': '16/12/18',
 'datetime': '2018-12-16T15:38:39',
 'lati

In [19]:
# h) The surface temperature is only applicable on days that there were fires.
# so we can average the values for surface temperature on the days there were fires.
h_answer = the_collection.aggregate([
    
    {"$match": {"latitude": {"$ne": None}}},
    {"$group": {"_id": "$date", "avg": {"$avg": "$surface_temperature_celsius"}}}])
for item in h_answer:
    pprint(item)

{'_id': '27/12/18', 'avg': 62.75}
{'_id': '25/12/18', 'avg': 54.0}
{'_id': '24/12/18', 'avg': 32.0}
{'_id': '16/12/18', 'avg': 57.8}
{'_id': '14/12/18', 'avg': 70.0}
{'_id': '13/12/18', 'avg': 60.0}
{'_id': '12/12/18', 'avg': 44.0}
{'_id': '21/10/18', 'avg': 51.25}
{'_id': '27/10/18', 'avg': 50.4}
{'_id': '15/10/18', 'avg': 72.66666666666667}
{'_id': '21/12/18', 'avg': 46.0}
{'_id': '23/10/18', 'avg': 38.0}
{'_id': '20/10/18', 'avg': 50.0}
{'_id': '12/11/18', 'avg': 53.0}
{'_id': '13/11/18', 'avg': 47.0}
{'_id': '16/10/18', 'avg': 36.0}
{'_id': '23/11/18', 'avg': 58.8}
{'_id': '30/11/18', 'avg': 52.41935483870968}
{'_id': '10/10/18', 'avg': 53.333333333333336}
{'_id': '28/10/18', 'avg': 56.0}
{'_id': '22/11/18', 'avg': 61.5}
{'_id': '17/10/18', 'avg': 51.6}
{'_id': '21/11/18', 'avg': 59.0}
{'_id': '15/12/18', 'avg': 39.0}
{'_id': '18/10/18', 'avg': 52.166666666666664}
{'_id': '11/11/18', 'avg': 46.25}
{'_id': '28/11/18', 'avg': 42.0}
{'_id': '26/10/18', 'avg': 44.6}
{'_id': '14/11/18',

In [20]:
# i) Here we can sort based on GHI and then limit to the first 10 entries.
i_answer = the_collection.find().sort("GHI_w/m2", pymongo.ASCENDING).limit(10)
for item in i_answer:
    pprint(item)

{'GHI_w/m2': 47.0,
 '_id': ObjectId('60aa1920c946a2389d1838ee'),
 'air_temperature_celsius': 5.0,
 'confidence': None,
 'date': '2/8/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_wind_speed': 5.1,
 'precipitation_level': 0.0,
 'precipitation_type': 'I',
 'relative_humidity': 38.6,
 'station': 948701.0,
 'surface_temperature_celsius': None,
 'windspeed_knots': 1.8}
{'GHI_w/m2': 48.0,
 '_id': ObjectId('60aa1920c946a2389d1838cd'),
 'air_temperature_celsius': 5.0,
 'confidence': None,
 'date': '30/6/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_wind_speed': 11.1,
 'precipitation_level': 0.0,
 'precipitation_type': 'A',
 'relative_humidity': 34.9,
 'station': 948701.0,
 'surface_temperature_celsius': None,
 'windspeed_knots': 4.3}
{'GHI_w/m2': 49.0,
 '_id': ObjectId('60aa1920c946a2389d1838ce'),
 'air_temperature_celsius': 5.0,
 'confidence': None,
 'date': '1/7/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_wind_speed': 14.

In [21]:
# j) Here we can firstly see if the precipitation was a 24 hour reading i.e there was a "G" in it'
# Then we can check if the precipitation was in the range of 0.2 to 0.35
j_answer = the_collection.find({"$and": [
    {"precipitation_type": {"$regex": "G", "$options": "$i"}},
    {"precipitation_level": {"$gt": 0.2, "$lt": 0.35}}   
]})
for item in j_answer:
    pprint(item)

{'GHI_w/m2': 157.0,
 '_id': ObjectId('60aa1920c946a2389d183825'),
 'air_temperature_celsius': 19.0,
 'confidence': None,
 'date': '13/1/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_wind_speed': 18.1,
 'precipitation_level': 0.31,
 'precipitation_type': 'G',
 'relative_humidity': 54.1,
 'station': 948700.0,
 'surface_temperature_celsius': None,
 'windspeed_knots': 11.2}
{'GHI_w/m2': 146.0,
 '_id': ObjectId('60aa1920c946a2389d183870'),
 'air_temperature_celsius': 17.0,
 'confidence': None,
 'date': '29/3/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_wind_speed': 21.0,
 'precipitation_level': 0.24,
 'precipitation_type': 'G',
 'relative_humidity': 49.9,
 'station': 948701.0,
 'surface_temperature_celsius': None,
 'windspeed_knots': 12.2}
{'GHI_w/m2': 166.0,
 '_id': ObjectId('60aa1920c946a2389d183886'),
 'air_temperature_celsius': 20.0,
 'confidence': None,
 'date': '20/4/18',
 'datetime': None,
 'latitude': None,
 'longitude': None,
 'max_win